[View in Colaboratory](https://colab.research.google.com/github/gauravbansal98/Sentiment-Classification-on-10k-samples/blob/master/Sentiment_Classification_on_10k_samples.ipynb)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import nltk
import matplotlib as plt
import os
os.chdir('drive/sentiment classification on 10k examples using rnn')

In [0]:
import nltk
nltk.download()

In [0]:
from collections import Counter
import random
import time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import keras
from keras.preprocessing.sequence import pad_sequences

lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
print(stop_words)

In [0]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren']

In [0]:
def create_lexicon(pos, neg):
  lexicon = []
  with open(pos, 'r') as f:
    lines = f.readlines()
    for line in lines:
      line = line.lower()
      words = word_tokenize(line)
      lexicon += words
  
  print(len(lexicon))
  with open(neg, 'r') as f:
    lines = f.readlines()
    for line in lines:
      line = line.lower()
      words = word_tokenize(line)
      lexicon += words
  
  print(len(lexicon))
  lexicon = [lemmatizer.lemmatize(word) for word in lexicon]
  
  lexicon_without_stop_words = []
  
  for word in lexicon:
    if word in stop_words:
      continue
    else:
      lexicon_without_stop_words.append(word)
  
  print(len(lexicon_without_stop_words))
  lexicon = nltk.FreqDist(lexicon_without_stop_words)
  
  final_lexicon = []

  for word in lexicon:
   
    if 5 < lexicon[word] and len(word) > 2 :
      final_lexicon.append(word)
      
  return final_lexicon

In [8]:
lexicon = create_lexicon('pos.txt', 'neg.txt')

115378
230193
147281


In [9]:
lexicon.append('0.0')
lexicon.append('123')
lexicon.append('321')
print(len(lexicon))
print(lexicon)

3467
['rock', 'destined', '21st', 'century', 'new', 'going', 'make', 'splash', 'even', 'greater', 'arnold', 'schwarzenegger', 'van', 'steven', 'gorgeously', 'elaborate', 'ring', 'huge', 'word', 'not', 'describe', 'peter', 'jackson', 'vision', 'effective', 'biopic', 'sometimes', 'like', 'movie', 'fun', 'good', 'place', 'start', 'emerges', 'something', 'rare', 'issue', 'honest', 'doe', "n't", 'feel', 'one', 'film', 'provides', 'great', 'insight', 'mindset', 'comic', 'absolute', 'top', 'game', 'offer', 'combination', 'entertainment', 'education', 'perhaps', 'picture', 'ever', 'made', 'literally', 'road', 'hell', 'intention', 'steer', 'turn', 'snappy', 'screenplay', 'edge', 'clever', 'want', 'hate', 'somehow', 'pull', 'take', 'care', 'cat', 'refreshingly', 'different', 'slice', 'asian', 'cinema', 'well', 'worth', 'seeing', 'talking', 'singing', 'head', 'really', 'surprise', 'low-key', 'quality', 'genuine', 'wendigo', 'eye', 'heart', 'mind', 'greatest', 'ultimately', 'reason', 'need', 'stor

In [0]:
pos_file = open('pos.txt', 'r')
pos_file_lines = pos_file.readlines()

neg_file = open('neg.txt', 'r')
neg_file_lines = neg_file.readlines()

pos_file.close()
neg_file.close()

In [0]:
def convert_to_list(file):
  list_of_eg = []
  for line in file:
    line = str(line)
    line= line.lower()
    words = word_tokenize(line)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    list_of_eg.append(lemmatized_words)
  #print('going out of convert_to_list')
  return list_of_eg

In [0]:
def padding_process(lines):
  
  padded_lines = pad_sequences(lines, maxlen = 30, dtype = 'str', padding = 'post', truncating = 'post')

  return padded_lines

In [0]:
def make_feature_vector(file):
  
  len_of_lexicon = len(lexicon)
  no_of_samples = len(file)
  feature_vector = []
  for line in file:
    features_of_line = []
    for word in line:
      one_hot_vector = np.zeros((len_of_lexicon))
      if word in lexicon :
        index_in_lexicon = lexicon.index(word)
        one_hot_vector[index_in_lexicon] = 1
      
      features_of_line.append(one_hot_vector)
    feature_vector.append(features_of_line)
  
  feature_vector = np.array(feature_vector)
  feature_vector = np.reshape(feature_vector, (no_of_samples, 30, len_of_lexicon))
  
  return feature_vector

In [14]:
total_set = []
for line in pos_file_lines:
  line = " " +str(123) + " "  + line 
  line = [line]
  total_set.append(line)
for line in neg_file_lines:
  line = " " + str(321) + " " + line
  line = [line]
  total_set.append(line)
  
np.random.shuffle(total_set)

training_set = total_set[:8960]
test_set = total_set[8960:10624]

training_set = list(training_set)
print(training_set[0])

print(word_tokenize(str(training_set[0])))
  

[" 321 it's of the quality of a lesser harrison ford movie - six days , seven nights , maybe , or that dreadful sabrina remake . \n"]
['[', '``', '321', 'it', "'s", 'of', 'the', 'quality', 'of', 'a', 'lesser', 'harrison', 'ford', 'movie', '-', 'six', 'days', ',', 'seven', 'nights', ',', 'maybe', ',', 'or', 'that', 'dreadful', 'sabrina', 'remake', '.', '\\n', "''", ']']


In [0]:
os.chdir('/content')
!git clone https://github.com/mixuala/colab_utils
import colab_utils.tboard
ROOT = %pwd
print(ROOT)
LOG_DIR = os.path.join(ROOT, 'gaurav')
print(LOG_DIR)
colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR )

In [0]:
from tensorflow.python.ops import rnn, rnn_cell
tf.reset_default_graph()


In [0]:
hm_epochs = 2
n_classes = 2
batch_size = 64
chunk_size = len(lexicon) - 2
n_chunks = 27
rnn_size = 64

In [0]:
x = tf.placeholder('float', [None, n_chunks,chunk_size], name = 'input labels')
y = tf.placeholder('float', name = 'output labels')
dropout_prob = tf.placeholder('float', name = 'dropout value')

layer = {'weights':tf.Variable(tf.random_normal([rnn_size,n_classes], name = 'weights')),
         'biases':tf.Variable(tf.random_normal([n_classes], name = 'biases'))}

In [0]:
def recurrent_neural_network(x):
  with tf.name_scope('neural_network'):
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0)

    lstm_cell = rnn_cell.BasicLSTMCell(rnn_size,state_is_tuple=True)
    lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell, output_keep_prob=dropout_prob)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    

    output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

    return output

In [36]:
def train_neural_network(x):
  len_of_lexicon = len(lexicon)
  
  prediction = recurrent_neural_network(x)
  with tf.name_scope('cost'):  
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y) )
  summary = tf.summary.scalar('total_cost_of_batch', cost)
  
  with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
  merged = tf.summary.merge_all()  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./gaurav',sess.graph)
    k = 0
    for epoch in range(hm_epochs):
      epoch_loss = 0
      starting = 0
      end  = batch_size

      while end < 8960:
        lines = training_set[starting:end]
        lines = convert_to_list(lines)
        lines = padding_process(lines)
        feature_vector = make_feature_vector(lines)

        epoch_x = feature_vector[:, 3:, :len_of_lexicon - 2]

        epoch_y = feature_vector[:, 2, len_of_lexicon - 2: len_of_lexicon]

        epoch_y = epoch_y.reshape((64, 2))

        epoch_y = np.array(epoch_y)
        epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))

        _, c, m = sess.run([optimizer, cost, merged], feed_dict={x: epoch_x, y: epoch_y, dropout_prob: 0.5})
        epoch_loss += c
        
        starting = end
        end = end + batch_size
        
        train_writer.add_summary(m, k)
        k += 1

      print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
    

    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

    lines = convert_to_list(test_set)
    lines = padding_process(lines)
    feature_vector = make_feature_vector(lines)


    epoch_x = feature_vector[:, 3:, :len_of_lexicon - 2]
    epoch_y = feature_vector[:, 2, len_of_lexicon - 2: len_of_lexicon]

    epoch_y = epoch_y.reshape((1664, 2))        

    print('Accuracy:',accuracy.eval({x:epoch_x, y:epoch_y, dropout_prob : 1.0}))

train_neural_network(x)

Epoch 0 completed out of 2 loss: 94.40527608990669
Epoch 1 completed out of 2 loss: 65.5027725994587
Accuracy: 0.73257214


In [134]:
!cat /proc/meminfo

MemTotal:       13341832 kB
MemFree:          164452 kB
MemAvailable:    6782932 kB
Buffers:          155620 kB
Cached:          6438668 kB
SwapCached:            0 kB
Active:          8631060 kB
Inactive:        3617556 kB
Active(anon):    5824724 kB
Inactive(anon):    99916 kB
Active(file):    2806336 kB
Inactive(file):  3517640 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               280 kB
Writeback:             0 kB
AnonPages:       5654368 kB
Mapped:           803456 kB
Shmem:            270312 kB
Slab:             597360 kB
SReclaimable:     547920 kB
SUnreclaim:        49440 kB
KernelStack:        4144 kB
PageTables:        21740 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6670916 kB
Committed_AS:    7719548 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:      6144 kB
HugePages_Total: 